In [2]:
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds
# !pip install tensorflow_datasets

In [3]:
mnist_dataset , mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-train.tfrecord...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to C:\Users\13kau\tensorflow_datasets\mnist\3.0.1. Subsequent calls will reuse this data.


In [4]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

In [7]:
# vaidate of 10%
num_valid_samples =0.1 * mnist_info.splits['train'].num_examples
num_valid_samples =tf.cast(num_valid_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

def scale(image, label):
    image = tf.cast(image, tf.float32)
    image /= 255.
    return image, label

scaled_train_and_validation_data = mnist_train.map(scale)    
test_data = mnist_test.map(scale)

BUFFER_SIZE = 10000

# Shuffling the data
shuffl_train_valid_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

validataion_data = scaled_train_and_validation_data.take(num_valid_samples)
train_data = scaled_train_and_validation_data.skip(num_valid_samples)

BATCH_SIZE = 100

train_data = train_data.batch(BATCH_SIZE)
validataion_data = validataion_data.batch(num_valid_samples)
test_data = test_data.batch(num_test_samples)


validation_inputs, validatin_targets = next(iter(validataion_data))

## Model

### Outline the model

In [14]:
input_size = 784
output_size = 10
hidden_layer_size = 150

model = tf.keras.Sequential([
    # Flatten the images to vector
    tf.keras.layers.Flatten(input_shape = (28, 28, 1)),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
    tf.keras.layers.Dense(output_size, activation='softmax')
])

## Choose the optimizer and the loss function

In [15]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

## Training

In [16]:
NUM_EPOCHS = 5

model.fit(train_data, epochs= NUM_EPOCHS, validation_data=(validation_inputs, validatin_targets), verbose=2)

Epoch 1/5
540/540 - 4s - loss: 0.2961 - accuracy: 0.9154 - val_loss: 0.1627 - val_accuracy: 0.9502
Epoch 2/5
540/540 - 3s - loss: 0.1186 - accuracy: 0.9646 - val_loss: 0.1222 - val_accuracy: 0.9622
Epoch 3/5
540/540 - 3s - loss: 0.0794 - accuracy: 0.9767 - val_loss: 0.1070 - val_accuracy: 0.9658
Epoch 4/5
540/540 - 3s - loss: 0.0581 - accuracy: 0.9827 - val_loss: 0.0997 - val_accuracy: 0.9702
Epoch 5/5
540/540 - 3s - loss: 0.0432 - accuracy: 0.9874 - val_loss: 0.1005 - val_accuracy: 0.9712


## Test the model

In [18]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 [==============================] - 0s 240ms/step - loss: 0.0886 - accuracy: 0.9730


In [19]:
print('Test loss:{0:.2f}. Test accuracy : {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss:0.09. Test accuracy : 97.30%
